<p style="font-size:32px;text-align:center"><b><font color=DarkMagenta>Amazon Apparel Recommendations</font></b></p>
<p style="font-size:20px;text-align:center"><b><font color=DarkGoldenRod>Part 1</font></b></p>

<h3><font color=OrangeRed>Business Objective<font></h3>
- <p style="font-size:16px">Recommending similar items/products (apparel) to the given product (apparel) in any e-commerce websites based on Text and Image Features.</p>
- <p style="font-size:16px">Recommending similar apparel items to user. Its estimated that amazon’s 35% revenue is generated using product recommendations.</p>
    
<h3><font color=OrangeRed>Approach<font></h3>
- <p style="font-size:16px"><b>Content Based Recommendation:</b> As its name suggests, we do content based recommendation, means its based on Title text, Description text and Images.</p>   
    

<h3><font color=MediumVioletRed>Loading and Reading Data</font></h3>

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
os.chdir("D:/Amazon Apparel Recommendation")

In [2]:
import pandas as pd
#we have a json file which consists of all information about the products
#loading the data using pandas
apparel_data = pd.read_json("amazon_apparel_data.json")

In [3]:
print("NUMBER OF DATA-POINTS -",apparel_data.shape[0])
print("NUMBER OF FEATURES/COLUMNS -",apparel_data.shape[1])

NUMBER OF DATA-POINTS - 183138
NUMBER OF FEATURES/COLUMNS - 19


In [4]:
#each product/item has 19 features in the raw dataset
#prints column-names or feature-names
apparel_data.columns

Index(['sku', 'asin', 'product_type_name', 'formatted_price', 'author',
       'color', 'brand', 'publisher', 'availability', 'reviews',
       'large_image_url', 'availability_type', 'small_image_url',
       'editorial_review', 'title', 'model', 'medium_image_url',
       'manufacturer', 'editorial_reivew'],
      dtype='object')

<p style="font-size:16px">Of these 19 features, we will use only 7 features.</p>
<p style="font-size:14px">1. asin (Amazon Standard Identification Number)<br>
    2. brand (Brand to which the product belongs to)<br>
    3. color (Color information of apparel, it can contain many colors as a value ex: red and black stripes)<br>
    4. product_type_name (Type of the apperal, ex: SHIRT/TSHIRT)<br>
    5. medium_image_url (url of the image)<br>
    6. title (Title of the product)<br>
    7. formatted_price (Price of the product)</p>

In [5]:
apparel_data = apparel_data[['asin', 'brand', 'color', 'medium_image_url', 'product_type_name', 'title', 'formatted_price']]

In [6]:
print("NUMBER OF DATA-POINTS -",apparel_data.shape[0])
print("NUMBER OF FEATURES/COLUMNS -",apparel_data.shape[1])

NUMBER OF DATA-POINTS - 183138
NUMBER OF FEATURES/COLUMNS - 7


In [7]:
apparel_data.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,B016I2TS4W,FNC7C,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,Minions Como Superheroes Ironman Long Sleeve R...,None
1,B01N49AI08,FIG Clothing,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Izo Tunic,None
2,B01JDPCOHO,FIG Clothing,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Won Top,None
3,B01N19U5H5,Focal18,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,Focal18 Sailor Collar Bubble Sleeve Blouse Shi...,None
4,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,Featherlite Ladies' Long Sleeve Stain Resistan...,$26.26


<p style="font-size:15px"><b><font color=SaddleBrown>Basic Stats For Feature -</font> product_type_name</b></p>

In [8]:
apparel_data['product_type_name'].describe()

#we have total 72 unique type of product type names
#91.62% (167794/183138) of the products are SHIRT

count     183138
unique        72
top        SHIRT
freq      167794
Name: product_type_name, dtype: object

In [9]:
#names of different product types
apparel_data['product_type_name'].unique()

array(['SHIRT', 'SWEATER', 'APPAREL', 'OUTDOOR_RECREATION_PRODUCT',
       'BOOKS_1973_AND_LATER', 'PANTS', 'HAT', 'SPORTING_GOODS', 'DRESS',
       'UNDERWEAR', 'SKIRT', 'OUTERWEAR', 'BRA', 'ACCESSORY',
       'ART_SUPPLIES', 'SLEEPWEAR', 'ORCA_SHIRT', 'HANDBAG',
       'PET_SUPPLIES', 'SHOES', 'KITCHEN', 'ADULT_COSTUME',
       'HOME_BED_AND_BATH', 'MISC_OTHER', 'BLAZER',
       'HEALTH_PERSONAL_CARE', 'TOYS_AND_GAMES', 'SWIMWEAR',
       'CONSUMER_ELECTRONICS', 'SHORTS', 'HOME', 'AUTO_PART',
       'OFFICE_PRODUCTS', 'ETHNIC_WEAR', 'BEAUTY',
       'INSTRUMENT_PARTS_AND_ACCESSORIES', 'POWERSPORTS_PROTECTIVE_GEAR',
       'SHIRTS', 'ABIS_APPAREL', 'AUTO_ACCESSORY', 'NONAPPARELMISC',
       'TOOLS', 'BABY_PRODUCT', 'SOCKSHOSIERY',
       'POWERSPORTS_RIDING_SHIRT', 'EYEWEAR', 'SUIT', 'OUTDOOR_LIVING',
       'POWERSPORTS_RIDING_JACKET', 'HARDWARE', 'SAFETY_SUPPLY',
       'ABIS_DVD', 'VIDEO_DVD', 'GOLF_CLUB', 'MUSIC_POPULAR_VINYL',
       'HOME_FURNITURE_AND_DECOR', 'TABLET_COMPUTER',

In [10]:
#find the 10 most frequent product_type_names
from collections import Counter

count_product_type = Counter(list(apparel_data['product_type_name']))
count_product_type.most_common(10)

[('SHIRT', 167794),
 ('APPAREL', 3549),
 ('BOOKS_1973_AND_LATER', 3336),
 ('DRESS', 1584),
 ('SPORTING_GOODS', 1281),
 ('SWEATER', 837),
 ('OUTERWEAR', 796),
 ('OUTDOOR_RECREATION_PRODUCT', 729),
 ('ACCESSORY', 636),
 ('UNDERWEAR', 425)]

<p style="font-size:15px"><b><font color=SaddleBrown>Basic Stats For Feature -</font> brand</b></p>

In [11]:
apparel_data['brand'].describe()

#there are 10577 unique brands
#183138 - 182987 = 151 missing values

count     182987
unique     10577
top         Zago
freq         223
Name: brand, dtype: object

In [12]:
#find the 10 most frequent brand names

count_brands = Counter(list(apparel_data['brand']))
count_brands.most_common(10)

[('Zago', 223),
 ('XQS', 222),
 ('Yayun', 215),
 ('YUNY', 198),
 ('XiaoTianXin-women clothes', 193),
 ('Generic', 192),
 ('Boohoo', 190),
 ('Alion', 188),
 ('Abetteric', 187),
 ('TheMogan', 187)]

<p style="font-size:15px"><b><font color=SaddleBrown>Basic Stats For Feature -</font> color</b></p>

In [13]:
apparel_data['color'].describe()

#there are 7380 unique colors
#183138 - 64956 = 118182 missing values, approx. 64.5% missing
#7.2% of products are black in color

count     64956
unique     7380
top       Black
freq      13207
Name: color, dtype: object

In [14]:
#find the 10 most frequent colors

count_colors = Counter(list(apparel_data['color']))
count_colors.most_common(10)

#118182 values are None (missing)

[(None, 118182),
 ('Black', 13207),
 ('White', 8616),
 ('Blue', 3570),
 ('Red', 2289),
 ('Pink', 1842),
 ('Grey', 1499),
 ('*', 1388),
 ('Green', 1258),
 ('Multi', 1203)]

<p style="font-size:15px"><b><font color=SaddleBrown>Basic Stats For Feature -</font> formatted_price</b></p>

In [15]:
apparel_data['formatted_price'].describe()

#183138 - 28395 = 154743 missing values, approx. 84.50% missing

count      28395
unique      3135
top       $19.99
freq         945
Name: formatted_price, dtype: object

In [16]:
#find the 10 most frequent price

count_prices = Counter(list(apparel_data['formatted_price']))
count_prices.most_common(10)

#154743 values are None (missing)

[(None, 154743),
 ('$19.99', 945),
 ('$9.99', 749),
 ('$9.50', 601),
 ('$14.99', 472),
 ('$7.50', 463),
 ('$24.99', 414),
 ('$29.99', 370),
 ('$8.99', 343),
 ('$9.01', 336)]

<p style="font-size:15px"><b><font color=SaddleBrown>Basic Stats For Feature -</font> title</b></p>

In [17]:
apparel_data['title'].describe()

#all of the products have a title
#titles are fairly descriptive of what the product is
#we use titles extensively in this case_study as they are short and informative

count                                                183138
unique                                               175985
top       Nakoda Cotton Self Print Straight Kurti For Women
freq                                                     77
Name: title, dtype: object

<p style="font-size:15px"><b><font color=SaddleBrown>Save Data Files</font></b></p>
<p style="font-size:14px">It is always good to save data files at every major step in our processing.<br>If we stuck anywhere (or) If some code takes too long to run on system, we may use the saved files to speed things up.</p>

In [18]:
import pickle
apparel_data.to_pickle("Saved Files/180k_Products_Data")

<h3><font color=MediumVioletRed>Data Cleaning - Transforming Data</font></h3>

<p style="font-size:15px"><b><font color=SaddleBrown>Eliminating Null Values</font></b></p>

In [19]:
#we are only considering the products which have price information
#apparel_data['formatted_price'].isnull() => gives us the information about the df row's which have null (None) price values 
#there are 154743 null (None) values in formatted_price feature
#so we will eliminating 154743 rows

apparel_data = apparel_data.loc[~apparel_data['formatted_price'].isnull()]
print("NUMBER OF DATA POINTS AFTER ELIMINATING PRICE = NULL -",apparel_data.shape[0])

NUMBER OF DATA POINTS AFTER ELIMINATING PRICE = NULL - 28395


In [20]:
#again we are considering the products which have color information
#apparel_data['color'].isnull() => gives us the information about the df row's which have null (None) color values

apparel_data = apparel_data.loc[~apparel_data['color'].isnull()]
print("NUMBER OF DATA POINTS AFTER ELIMINATING COLOR = NULL -",apparel_data.shape[0])

NUMBER OF DATA POINTS AFTER ELIMINATING COLOR = NULL - 28385


<p style="font-size:15px"><b><font color=SeaGreen>We brought down the number of data points from 183K to 28K.</font></b></p>

In [21]:
#save the data
apparel_data.to_pickle("Saved Files/28k_Products_Data")

<p style="font-size:15px"><b><font color=SaddleBrown>Download Images For Products From Provided URLs</font></b></p>

In [22]:
#we can download all these 28k images using this code below

from PIL import Image
import requests
from io import BytesIO

for index, row in images.iterrows():
        url = row['large_image_url']
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img.save('Saved Files/28k_Images/'+row['asin']+'.jpeg')

<p style="font-size:15px"><b><font color=SaddleBrown>Remove Duplicate Items</font></b></p>

In [23]:
#read the data from pickle file as we saved in previous stage
apparel_data = pd.read_pickle("Saved Files/28k_Products_Data")

In [24]:
apparel_data.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
4,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,Featherlite Ladies' Long Sleeve Stain Resistan...,$26.26
6,B012YX2ZPI,HX-Kingdom Fashion T-shirts,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,Women's Unique 100% Cotton T - Special Olympic...,$9.99
11,B001LOUGE4,Fitness Etc.,Black,https://images-na.ssl-images-amazon.com/images...,SHIRT,Ladies Cotton Tank 2x1 Ribbed Tank Top,$11.99
15,B003BSRPB0,FeatherLite,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,FeatherLite Ladies' Moisture Free Mesh Sport S...,$20.54
21,B014ICEDNA,FNC7C,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,Supernatural Chibis Sam Dean And Castiel Short...,$7.50


In [25]:
#find number of products that have duplicate titles
sum(apparel_data.duplicated('title'))

#we have 2325 products which have same title

2325

![](https://gitlab.com/akashbangalkar/amazon-apparel-recommendations-system/-/raw/main/Sample1.png)
![](https://gitlab.com/akashbangalkar/amazon-apparel-recommendations-system/-/raw/main/Sample2.png)

- <p style="font-size:15px">Given a query image, recommending exactly the same product slightly change in color or size is **not the best recommendation or suggestion**.</p>
- <p style="font-size:15px">In our data there are many duplicate products like the above examples, we need to **de-dupe (removing duplicates)** them for the better results.</p>

<p style="font-size:15px"><b><font color=DarkTurquoise>Remove Duplicate Items (Part 1)</font></b></p>

In [26]:
#remove all products with very few words in title
#i.e., removing all the title with less than 5 words
sorted_apparel_data = apparel_data[apparel_data['title'].apply(lambda x: len(x.split())>4)]
print("AFTER REMOVAL OF PRODUCTS WITH SHORT DESCRIPTION:",sorted_apparel_data.shape[0])

AFTER REMOVAL OF PRODUCTS WITH SHORT DESCRIPTION: 27949


In [27]:
#Sort the whole data based on title in alphabetical order
sorted_apparel_data.sort_values('title', inplace=True, ascending=False)
sorted_apparel_data.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
61973,B06Y1KZ2WB,Éclair,Black/Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,Éclair Women's Printed Thin Strap Blouse Black...,$24.99
133820,B010RV33VE,xiaoming,Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Womens Sleeveless Loose Long T-shirts...,$18.19
81461,B01DDSDLNS,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Women's White Long Sleeve Single Brea...,$21.58
75995,B00X5LYO9Y,xiaoming,Red Anchors,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Stripes Tank Patch/Bear Sleeve Anchor...,$15.91
151570,B00WPJG35K,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Sleeve Sheer Loose Tassel Kimono Woma...,$14.32


<p style="font-size:15px">Some examples of dupliacte titles that differ only in the last few words.</p>
<pre>
Titles 1:
16. woman's place is in the house and the senate shirts for Womens XXL White
17. woman's place is in the house and the senate shirts for Womens M Grey

Title 2:
25. tokidoki The Queen of Diamonds Women's Shirt X-Large
26. tokidoki The Queen of Diamonds Women's Shirt Small
27. tokidoki The Queen of Diamonds Women's Shirt Large

Title 3:
61. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
62. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
63. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
64. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
</pre>

In [28]:
indices = []
for i, row in sorted_apparel_data.iterrows():
    indices.append(i)

In [29]:
import itertools

dedupe_asins = []
i = 0
j = 0

num_data_points = sorted_apparel_data.shape[0]

while (i < num_data_points) and (j < num_data_points):
    
    previous_i = i
    #store the list of words of ith string in a
    #ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    a = sorted_apparel_data['title'].loc[indices[i]].split()

    #search for the similar products sequentially 
    j = i+1
    while (j < num_data_points):

        #store the list of words of jth string in b
        #ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'Small']
        b = sorted_apparel_data['title'].loc[indices[j]].split()

        #store the maximum length of two strings
        length = max(len(a), len(b))

        #count is used to store the number of words that are matched in both strings
        count  = 0

        #itertools.zip_longest(a,b): will map the corresponding words in both strings, 
        #it will appened None in case of unequal strings
        #example: a =['a', 'b', 'c', 'd']
        #b = ['a', 'b', 'd']
        
        #itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1

        #if the number of words in which both strings differ are > 2 , we are considering it as those two apperals are different
        #if the number of words in which both strings differ are < 2 , we are considering it as those two apperals are same, 
        #hence we are ignoring them
        if (length - count) > 2: #number of words in which both sentences differ
            #if both strings are differ by more than 2 words we include the 1st string index
            dedupe_asins.append(sorted_apparel_data['asin'].loc[indices[i]])

            #start searching for similar apperals corresponds 2nd string
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break

In [30]:
apparel_data = sorted_apparel_data.loc[sorted_apparel_data['asin'].isin(dedupe_asins)]

#we removed the dupliactes which differ only at the end

In [31]:
print("NUMBER OF DATA POINTS AFTER REMOVING DUPLICATES WHICH DIFFER ONLY AT THE END -",apparel_data.shape[0])

NUMBER OF DATA POINTS AFTER REMOVING DUPLICATES WHICH DIFFER ONLY AT THE END - 17592


In [32]:
#save the data
apparel_data.to_pickle("Saved Files/17k_Products_Data")

<p style="font-size:15px"><b><font color=DarkTurquoise>Remove Duplicate Items (Part 2)</font></b></p>

<p style="font-size:15px">In the previous cell, we sorted whole data in alphabetical order of  titles. Then, we removed titles which are adjacent and very similar title.<br>
<br>
But there are some products whose titles are not adjacent but very similar.</p>
<pre>
Examples:

Titles-1
86261.  UltraClub Women's Classic Wrinkle-Free Long Sleeve Oxford Shirt, Pink, XX-Large
115042. UltraClub Ladies Classic Wrinkle-Free Long-Sleeve Oxford Light Blue XXL

TItles-2
75004.  EVALY Women's Cool University Of UTAH 3/4 Sleeve Raglan Tee
109225. EVALY Women's Unique University Of UTAH 3/4 Sleeve Raglan Tees
120832. EVALY Women's New University Of UTAH 3/4-Sleeve Raglan Tshirt
</pre>

In [33]:
#read the data from pickle file as we saved in previous stage
apparel_data = pd.read_pickle("Saved Files/17k_Products_Data")

In [34]:
#this code snippet takes significant amount of time
#O(n^2) time.
#it takes about an hour to run on a decent system

indices = []
for i,row in apparel_data.iterrows():
    indices.append(i)

dedupe_asins = []

while len(indices)!=0:
    i = indices.pop()
    dedupe_asins.append(apparel_data['asin'].loc[i])
    
    #consider the first apperal's title
    a = apparel_data['title'].loc[i].split()
    
    #store the list of words of ith string in a
    #ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    for j in indices:
        
        b = apparel_data['title'].loc[j].split()
        #store the list of words of jth string in b
        #ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
        
        length = max(len(a),len(b))
        
        #count is used to store the number of words that are matched in both strings
        count  = 0

        #itertools.zip_longest(a,b): will map the corresponding words in both strings,
        #it will appened None in case of unequal strings
        #example: a =['a', 'b', 'c', 'd']
        #b = ['a', 'b', 'd']
        #itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0]==k[1]):
                count += 1

        #if the number of words in which both strings differ are < 3, 
        #we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) < 3:
            indices.remove(j)

In [35]:
#from whole previous products we will consider only the products that are found in previous cell 
apparel_data = apparel_data.loc[apparel_data['asin'].isin(dedupe_asins)]

In [37]:
print("NUMBER OF DATA POINTS AFTER REMOVING DUPLICATES -",apparel_data.shape[0])

NUMBER OF DATA POINTS AFTER REMOVING DUPLICATES - 16042


In [38]:
#save the data
apparel_data.to_pickle("Saved Files/16k_Products_Data")
#storing these products in a pickle file

<p style="font-size:20px;text-align:center"><b><font color=DarkSlateGray>For Text Preprocessing And Product Similarity - Refer <font color=DeepPink><a href="https://gitlab.com/akashbangalkar/amazon-apparel-recommendations-system/-/blob/main/2._Amazon_Apparel_Recommendations.ipynb?expanded=true&viewer=rich">Part 2</a></font> Notebook</font></b></p>